In [173]:
using CSV, DataFrames
using FileIO, Dates
using Graphs, MetaGraphs

include("../region_cube_split/cubes.jl")

region_cube_split (generic function with 1 method)

In [174]:
romania_full = CSV.read("../data/romania.csv", DataFrame);
romania = romania_full[romania_full.Datetime .> DateTime(1976,1,1,0,0,0),:];

vrancea = romania[(romania.Latitude .>= 45.20) .& (romania.Latitude .<= 46.0) .& 
                    (romania.Longitude .>= 26.0) .& (romania.Longitude .<= 27.0) .&
                    (romania.Depth .>= 50.0) .& (romania.Depth .<= 200.0), :];

In [175]:
vrancea, vrancea_cubes = region_cube_split(vrancea,side=5);

In [4]:
# cube_dictionary = Dict()
# for i=1:length(vrancea_cubes.cubeIndex)
#     cube_dictionary[vrancea_cubes.cubeIndex[i]] = i
# end
# cube_dictionary;

In [216]:
G = Graphs.Graph(length(vrancea_cubes.cubeIndex))
MG = MetaGraph(G)

for i in 1:nv(MG)
    set_prop!(MG, i, :cubeIndex, vrancea_cubes.cubeIndex[i])
end

set_indexing_prop!(MG, :cubeIndex)

Set{Symbol} with 1 element:
  :cubeIndex

In [217]:
edgeWeight=true

true

In [218]:
i=0
while i < length(vrancea.cubeIndex)-1

    i+=1
    current_node = MG[vrancea.cubeIndex[i],:cubeIndex]
    target_node = MG[vrancea.cubeIndex[i+1],:cubeIndex]

    
    if edgeWeight == false
        add_edge!(MG,current_node,target_node)
    else
        if has_edge(MG, current_node, target_node)
            set_prop!(MG,current_node,target_node,
                    :weight, get_prop(MG,Edge(current_node,target_node),:weight)+1)
            continue

        else
            add_edge!(MG,current_node,target_node,:weight,1)
        end
    end

end

In [219]:
MG

{1727, 7612} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [234]:
G = Graph(MG)

{1727, 7612} undirected simple Int64 graph

In [235]:
for i in 1:nv(MG)
    set_prop!(MG, i, :xLatitude, vrancea_cubes.xLatitude[i])
    set_prop!(MG, i, :yLongitude, vrancea_cubes.yLongitude[i])
    set_prop!(MG, i, :zDepth, vrancea_cubes.zDepth[i])
end

In [237]:
props(MG,2)

Dict{Symbol, Any} with 4 entries:
  :zDepth     => 22
  :cubeIndex  => "6183"
  :xLatitude  => 10
  :yLongitude => 8

In [258]:
using GraphIO, EzXML

In [262]:
savegraph("./vrancea_5km.graphml", MG, GraphMLFormat())

1

In [248]:
savegraph("./vrancea_5km.txt",MG, EdgeListFormat())

1